In [ ]:
!pip install -U pip setuptools wheel
!pip install thinc==8.2.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.42.0
    Uninstalling wheel-0.42.0:
      Successfully uninstalled wheel-0.42.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.0.3
    Uninstalling setuptools-69.0.3:
      Successfully uninstalled setuptools-69.0.3
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
conda 24.3.0 requires packaging>=23.

In [ ]:
#use when running with GPU
# !nvcc --version
# import cupy
# cupy.show_config()
# !nvidia-smi

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

In [ ]:
nlp = spacy.load("en_core_web_lg")

Uses the output-combined3 from ner_skin_datagen

In [ ]:
data1 = []
file_path = '/kaggle/input/output-combined3/output-combined3.txt'
# Open the file in read mode
count = 0
with open(file_path, 'r') as file:
    # Read each line of the file and append it to the list
    for line in file:
        data1.append(eval(line))

In [ ]:
import random
def split_data(data, train_ratio):
    random.shuffle(data)
    num_train_samples = int(len(data) * train_ratio)
    train_data = data[:num_train_samples]
    test_data = data[num_train_samples:]
    return train_data, test_data


Splits data for testing and training

In [ ]:
train_data, test_data = split_data(data1, 0.9)

# Example: Print the sizes of the training and testing sets
print("Training data size:", len(train_data))
print("Testing data size:", len(test_data))

Training data size: 376504
Testing data size: 41834


Converts the dataset into spacy format suitable for training.

In [ ]:
def generate_spacy(file, file_name):
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(file): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span != None:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)

#os.chdir(r'XXXX\XXXXX')
    db.to_disk(file_name) # save the docbin object
    return



In [ ]:
generate_spacy(train_data, "./train.spacy")

100%|██████████| 376504/376504 [02:35<00:00, 2414.09it/s]


In [ ]:
generate_spacy(test_data, "./test.spacy")

100%|██████████| 41834/41834 [00:17<00:00, 2339.27it/s]


Required the baseconfig, and please update the path accordingly. Since I have used Kaggle, I had to import the config and use it's path.

In [ ]:
!python -m spacy init fill-config /kaggle/input/toke2vec-ner-batch8-6000-epoch/base_config_batch_8.cfg base.cfg

✔ Auto-filled config with all values
✔ Saved config
base.cfg
You can now add your data and train your pipeline:
python -m spacy train base.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
gpu = spacy.prefer_gpu()
print(gpu)

False


This is where the training happens, can take upto 4-5 hours in CPU.

In [ ]:
!python -m spacy train base.cfg  --output /kaggle/working/output --paths.train /kaggle/working/train.spacy --paths.dev /kaggle/working/train.spacy

ℹ Saving to output directory: /kaggle/working/output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2024-05-03 06:24:10,168] [INFO] Set up nlp object from config
[2024-05-03 06:24:10,229] [INFO] Pipeline: ['tok2vec', 'ner']
[2024-05-03 06:24:10,237] [INFO] Created vocabulary
[2024-05-03 06:24:10,237] [INFO] Finished initializing nlp object
[2024-05-03 06:30:58,070] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     29.50    2.13    1.96    2.34    0.02
  0     200         40.48   1324.68   70.16   72.45   68.00    0.70
  0     400         50.13    521.19   84.22   80.18   88.68    0.84
  0     600         51.76  

In [ ]:
cat output/model-best/meta.json


{
  "lang":"en",
  "name":"pipeline",
  "version":"0.0.0",
  "spacy_version":">=3.7.4,<3.8.0",
  "description":"",
  "author":"",
  "email":"",
  "url":"",
  "license":"",
  "spacy_git_version":"bff8725f4",
  "vectors":{
    "width":0,
    "vectors":0,
    "keys":0,
    "name":null,
    "mode":"default"
  },
  "labels":{
    "tok2vec":[

    ],
    "ner":[
      "Skin disease"
    ]
  },
  "pipeline":[
    "tok2vec",
    "ner"
  ],
  "components":[
    "tok2vec",
    "ner"
  ],
  "disabled":[

  ],
  "performance":{
    "ents_f":0.9248622146,
    "ents_p":0.8700365492,
    "ents_r":0.9870622997,
    "ents_per_type":{
      "Skin disease":{
        "p":0.8700365492,
        "r":0.9870622997,
        "f":0.9248622146
      }
    },
    "tok2vec_loss":148.7275556877,
    "ner_loss":1794.68837273
  }
}

In [ ]:
!zip -r out-cpu.zip /kaggle/working/output/

  adding: kaggle/working/output/ (stored 0%)
  adding: kaggle/working/output/model-last/ (stored 0%)
  adding: kaggle/working/output/model-last/vocab/ (stored 0%)
  adding: kaggle/working/output/model-last/vocab/key2row (stored 0%)
  adding: kaggle/working/output/model-last/vocab/lookups.bin (stored 0%)
  adding: kaggle/working/output/model-last/vocab/strings.json (deflated 74%)
  adding: kaggle/working/output/model-last/vocab/vectors.cfg (stored 0%)
  adding: kaggle/working/output/model-last/vocab/vectors (deflated 45%)
  adding: kaggle/working/output/model-last/config.cfg (deflated 61%)
  adding: kaggle/working/output/model-last/meta.json (deflated 57%)
  adding: kaggle/working/output/model-last/tok2vec/ (stored 0%)
  adding: kaggle/working/output/model-last/tok2vec/cfg (stored 0%)
  adding: kaggle/working/output/model-last/tok2vec/model (deflated 8%)
  adding: kaggle/working/output/model-last/tokenizer (deflated 81%)
  adding: kaggle/working/output/model-last/ner/ (stored 0%)
  addi

For test the model. You can add the model and substitute it's path here to test it directly.

In [ ]:
!python -m spacy benchmark accuracy /kaggle/working/output/model-best /kaggle/working/test.spacy

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   86.90 
NER R   98.69 
NER F   92.42 
SPEED   11812 


=============================== NER (per type) ===============================

                   P       R       F
Skin disease   86.90   98.69   92.42



In [ ]:
nlp2 = spacy.load(r"/kaggle/input/best-model/kaggle/working/output/model-best") #load the best model
doc = nlp2("Skin diseases encompass a wide range of conditions that affect the largest organ of the human body. These diseases can manifest in various forms, from minor irritations to severe and chronic ailments. Acne keloidalis nuchae, characterized by inflammation and itching, is one of the most common skin conditions worldwide. Psoriasis, a chronic autoimmune disease, leads to the rapid buildup of skin cells, resulting in red, scaly patches. Acne vulgaris, often seen in adolescents, occurs due to blockage of hair follicles by oil and dead skin cells, leading to pimples and cysts. Eczema, also known as atopic dermatitis, causes dry, itchy patches on the skin and is often linked to allergies and asthma. Rosacea is a chronic condition characterized by facial redness, swelling, and small, pus-filled bumps. Vitiligo is a pigmentation disorder that causes white patches to develop on the skin due to the loss of melanin-producing cells. Impetigo, a highly contagious bacterial infection, leads to red sores that rupture and form honey-colored crusts. Shingles, caused by the varicella-zoster virus, results in painful rashes and blisters along nerve pathways. Dermatitis herpetiformis is a chronic blistering skin condition associated with gluten intolerance and is characterized by intensely itchy, red bumps and blisters. Finally, melanoma, the deadliest form of skin cancer, develops in the pigment-producing cells of the skin and can metastasize to other parts of the body if not detected and treated early.") # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) 

Now we can use the evalutation samples and check it's metric on the model. Uses the eval.txt.

In [ ]:
data2 = []
file_path = '/kaggle/input/evaluation/eval.txt'
# Open the file in read mode
count = 0
with open(file_path, 'r') as file:
    # Read each line of the file and append it to the list
    for line in file:
        data2.append(eval(line))

In [ ]:
from spacy.scorer import Scorer
from spacy.training import Example

scorer = Scorer()
examples = []
for text, annotations in test_data:
    doc_pred = nlp2(text)
    example = Example.from_dict(doc_pred, annotations)
    examples.append(example)
scores = scorer.score(examples)

In [ ]:
scores

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'sents_p': None,
 'sents_r': None,
 'sents_f': None,
 'tag_acc': None,
 'pos_acc': None,
 'morph_acc': None,
 'morph_micro_p': None,
 'morph_micro_r': None,
 'morph_micro_f': None,
 'morph_per_feat': None,
 'dep_uas': None,
 'dep_las': None,
 'dep_las_per_type': None,
 'ents_p': 0.684161456619502,
 'ents_r': 0.9666046799938013,
 'ents_f': 0.8012202954399485,
 'ents_per_type': {'Skin disease': {'p': 0.684161456619502,
   'r': 0.9666046799938013,
   'f': 0.8012202954399485}},
 'cats_score': 0.0,
 'cats_score_desc': 'macro F',
 'cats_micro_p': 0.0,
 'cats_micro_r': 0.0,
 'cats_micro_f': 0.0,
 'cats_macro_p': 0.0,
 'cats_macro_r': 0.0,
 'cats_macro_f': 0.0,
 'cats_macro_auc': 0.0,
 'cats_f_per_type': {},
 'cats_auc_per_type': {}}

In [ ]:
generate_spacy(data2, "./evaluation.spacy")

In [ ]:
from spacy.scorer import Scorer
from spacy.training import Example

scorer = Scorer()
examples = []
for text, annotations in data2:
    doc_pred = nlp2(text)
    example = Example.from_dict(doc_pred, annotations)
    examples.append(example)
scores = scorer.score(examples)

In [ ]:
scores

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'sents_p': None,
 'sents_r': None,
 'sents_f': None,
 'tag_acc': None,
 'pos_acc': None,
 'morph_acc': None,
 'morph_micro_p': None,
 'morph_micro_r': None,
 'morph_micro_f': None,
 'morph_per_feat': None,
 'dep_uas': None,
 'dep_las': None,
 'dep_las_per_type': None,
 'ents_p': 0.6386554621848739,
 'ents_r': 0.608,
 'ents_f': 0.6229508196721311,
 'ents_per_type': {'Skin disease': {'p': 0.6386554621848739,
   'r': 0.608,
   'f': 0.6229508196721311}},
 'cats_score': 0.0,
 'cats_score_desc': 'macro F',
 'cats_micro_p': 0.0,
 'cats_micro_r': 0.0,
 'cats_micro_f': 0.0,
 'cats_macro_p': 0.0,
 'cats_macro_r': 0.0,
 'cats_macro_f': 0.0,
 'cats_macro_auc': 0.0,
 'cats_f_per_type': {},
 'cats_auc_per_type': {}}

In [32]:
!python -m spacy benchmark accuracy /kaggle/working/output/model-best /kaggle/working/evaluation.spacy

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   65.60 
NER R   65.60 
NER F   65.60 
SPEED   12996 


=============================== NER (per type) ===============================

                   P       R       F
Skin disease   65.60   65.60   65.60

